Я для себя искал вхождения через re.findall циклом по словам из блэклиста, но решил, что в проде нужно чтобы работало быстрее, поэтому добавил в блэклисты побольше форм слов (прилагательные, глаголы) и поместил их в множества, где поиск О(1).
Может вообще зря парился и основная времязатрата здесь pymorphy

In [25]:
import os
import os.path
import json
import nltk
import re
#!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

### Подгружаем стоп-листы из текстовых файлов
def get_stop_lists():
    drug_words = []
    with open ('drugs.txt', 'r', encoding = 'utf-8') as f:
        for line in f.readlines():
            drug_words.append(line[:-1])
    suicide_words = []
    with open ('suicide.txt', 'r', encoding = 'utf-8') as f:
        for line in f.readlines():
            suicide_words.append(line[:-1])
    suicide_bigrams = []
    with open ('suicide_bigrams.txt', 'r', encoding = 'utf-8') as f:
        for line in f.readlines():
            suicide_bigrams.append(line[:-1])
    lgbt_words = []
    with open ('lgbt.txt', 'r', encoding = 'utf-8') as f:
        for line in f.readlines():
            lgbt_words.append(line[:-1])
    return (set(drug_words), set(suicide_words), set(suicide_bigrams), set(lgbt_words))

### Функция, проверяющая наличие среди нормированных токенов слов из стоп-листа
### и возвращающая их число, их порядковые номера, их форму из изначального текста
def check_unigrams(tokens, tokens_normed, begin, end, blacklist):
    bad_indexes = []
    bad_words = []
    score = 0
    for i in range(begin, end):
        if tokens_normed[i] in blacklist:
            score+=1
            bad_indexes.append(i)
            bad_words.append(tokens[i])
    return {'score':score, 'bad_indexes':bad_indexes, 'bad_words':bad_words}
            
def check_bigrams(tokens, tokens_normed, begin, end, blacklist):
    bad_indexes = []
    bad_words = []
    score = 0
    for i in range(begin, end-1):
        if tokens_normed[i]+' '+tokens_normed[i+1] in blacklist:
            score+=1
            bad_indexes.append(i)
            bad_indexes.append(i+1)
            bad_words.append(tokens[i])
            bad_words.append(tokens[i+1])
    return {'score':score, 'bad_indexes':bad_indexes, 'bad_words':bad_words}


### Главная функция, принимающая на вход json-файл из совы, бьщая его на блоки для анализа
### Проверяющая эти блоки на слова из стоп-списка
def check_text(json_file, drug_threshold = 2, lgbt_threshold = 1, suicide_threshold = 1, window_of_analysis = 100):
    drug_words, suicide_words, suicide_bigrams, lgbt_words = get_stop_lists()    
    tokens = nltk.word_tokenize(json_file['r'][0]['response'][0]['text'].lower())
    tokens_normed = []
    for token in tokens:
        tokens_normed.append(morph.parse(token)[0].normal_form)
    for low_border in range(0, len(tokens), window_of_analysis//2):
        up_border = min(low_border+window_of_analysis, len(tokens)-1)
        drug_report = check_unigrams(tokens, tokens_normed, low_border, up_border, drug_words)
        lgbt_report = check_unigrams(tokens, tokens_normed, low_border, up_border, lgbt_words)
        suicide_report = check_unigrams(tokens, tokens_normed, low_border, up_border, suicide_words)
        suicide_report2 = check_bigrams(tokens, tokens_normed, low_border, up_border, suicide_bigrams)
        if drug_report['score'] > drug_threshold:
            print(drug_report['bad_words'])
            #pass Здесь логика передающая репорт с указанием  begin, end, bad_words 
        if lgbt_report['score'] > lgbt_threshold:
            print(lgbt_report['bad_words'])
        if suicide_report['score'] + suicide_report2['score'] > suicide_threshold:
            print(suicide_report['bad_words'])
            print(suicide_report2['bad_words'])

In [24]:
cur_dir = os.getcwd()
path_sova_new = os.path.join(cur_dir, 'texts_sova_2')
filenames = []
with os.scandir(path_sova_new) as files:
    for file in files:
        filenames.append(file)
for file in filenames:
    if str(file)[-6:-2] == '.txt':
        with open (file, 'r', encoding = 'utf-8') as mfile:
            x = mfile.read()
            js = json.loads(x)
            check_text(js)

['нарко', 'кокаин', 'кокс', 'декстра', 'метамфетамин', 'гашиш', 'гаш', 'метадон', 'героин', 'герыч', 'гидропоника', 'шишка', 'альфа', 'вес', 'дозировка']
['героин', 'герыч', 'гидропоника', 'шишка', 'альфа', 'вес', 'дозировка']
['веществами', 'мефедрон', 'наркотиками']
['шишка', 'наркотики', 'мефедрона', 'наркотики']
['наркотики', 'мефедрона', 'наркотики', 'шишка']
['кокс', 'гашиш', 'нарко', 'экстази', 'трипа']
['кокс', 'гашиш', 'нарко', 'экстази', 'трипа', 'наркоману']
['наркотики', 'конопля', 'марихуана']
['наркотиков', 'конопли', 'гашиша', 'наркотик', 'марки']
['гашиша', 'наркотик', 'марки']
['гашиш', 'нарко', 'героин', 'гашиш', 'кокаин']
['нарко', 'героин', 'гашиш', 'кокаин', 'марихуана']
['шишки', 'нарко', 'наркотики', 'наркотики']
['наркотики', 'наркотики', 'кристаллы', 'траву', 'мефедрон']
['кристаллы', 'траву', 'мефедрон', 'травы', 'героина']
['нарко', 'вещество', 'героина']
['наркотика', 'шишки', 'трип', 'трава', 'гашиш']
['шишки', 'трип', 'трава', 'гашиш']
['нарко', 'наркотики